In [1]:
import warnings
warnings.filterwarnings("ignore", message=".*SQLAlchemy connectable.*")

In [2]:
import pandas as pd
import numpy as np
import re

#### Server Connection

In [3]:
import json, os
import pyodbc
with open(os.path.join(os.path.expanduser("~"), ".my_creds.json")) as f:
    creds = json.load(f)

db_user = creds["db_user"]
db_pass = creds["db_pass"]
server = "157.119.230.120,4071"
database = "PowerBI_LISV2"
#connection string
conn_str = f"""
DRIVER={{ODBC Driver 17 for SQL Server}};
SERVER={server};
DATABASE={database};
UID={db_user};
PWD={db_pass};
TrustServerCertificate=yes;
"""

conn = pyodbc.connect(conn_str)
print("Server Connected")

Server Connected


### StoreAttendance

In [4]:
StoreAttendanceQuery="""
SELECT UserName,FullName,LISStoreCode,StoreName,VisitDate,PJPStatus,[CheckIn-Time], [CheckOut-Time],TimeDuration,
    AutoCheckOutStatus,NoPermissionReason,[StoreCheck-InImageLink],[StoreCheck-OutImageLink],Null as 'FR Image',
    NoPermissionImageLink    
FROM PowerBI_LISV2.DaburPharmaManagement.ISPStoreAttendance_NormalExport
WHERE 
    UserName NOT LIKE '%SUP%'
    AND LOWER(UserName) NOT LIKE '%test%'
    AND LOWER(UserName) NOT LIKE '%star%'
    AND [Month] = 11
    AND [Year] = 2025
    AND TRY_CONVERT(date, [VisitDate], 105) >= '2025-11-01'
    AND TRY_CONVERT(date, [VisitDate], 105) <= '2025-11-05'
ORDER BY 
    [VisitDate],
    [CheckIn-Time];

"""

In [5]:
StoreAttendance= pd.read_sql(StoreAttendanceQuery,conn)

In [6]:
StoreAttendance.columns

Index(['UserName', 'FullName', 'LISStoreCode', 'StoreName', 'VisitDate',
       'PJPStatus', 'CheckIn-Time', 'CheckOut-Time', 'TimeDuration',
       'AutoCheckOutStatus', 'NoPermissionReason', 'StoreCheck-InImageLink',
       'StoreCheck-OutImageLink', 'FR Image', 'NoPermissionImageLink'],
      dtype='object')

In [7]:
# Convert to datetime.time format and back to string in HH:MM:SS
StoreAttendance["CheckIn-Time"] = pd.to_datetime(StoreAttendance["CheckIn-Time"], format="%H:%M:%S").dt.strftime("%H:%M:%S")
StoreAttendance["CheckOut-Time"] = pd.to_datetime(StoreAttendance["CheckOut-Time"], format="%H:%M:%S").dt.strftime("%H:%M:%S")

In [8]:
StoreAttendance["Covered"] = "Y"
StoreAttendance["Merchandised"] = np.where(
    StoreAttendance["NoPermissionReason"].isna() | (StoreAttendance["NoPermissionReason"] == ""),
    "Y",
    "N"
)

In [9]:
# Convert to datetime (if not already)
# StoreAttendance["VisitDate"] = pd.to_datetime(StoreAttendance["VisitDate"], errors="coerce")

# # Extract only the date part (keeps as datetime.date)
# StoreAttendance["VisitDate"] = StoreAttendance["VisitDate"].dt.date
# StoreAttendance["VisitDate"] = StoreAttendance["VisitDate"].apply(lambda x: x.strftime("%d-%m-%Y"))

In [10]:
StoreAttendance.to_csv('StoreAttendance.csv')

In [11]:
StoreAttendance.head()

UserName                        FullName          LISStoreCode  \
0  DBR-ISP430                  Turpaati Ramya   DBRPH-HYD0TG1705671   
1  DBR-ISP410                      B Surendra  DBRPH-NDY0AP00945235   
2  DBR-ISP440                     UMESH KUMAR  DBRPH-DEL0DL11616633   
3  DBR-ISP440                     UMESH KUMAR    DBRPH-DEL0DL894323   
4  DBR-ISP441  MUDASSIR MOULASAB GUNDAKARJIGI  DBRPH-MPS0GO11556272   

                                    StoreName            VisitDate PJPStatus  \
0  R S BROTHERS RETAILS INDIA PRIVATE LIMITED  01-11-2025 07:30:18   Planned   
1                     NEW BUDGET SUPER MARKET  01-11-2025 08:00:44   Planned   
2                                 AKASH STORE  01-11-2025 08:05:46   Planned   
3                                 SAW TRADERS  01-11-2025 08:16:39   Planned   
4                               AJ 7 SANGOLDA  01-11-2025 08:31:58   Planned   

  CheckIn-Time CheckOut-Time TimeDuration AutoCheckOutStatus  \
0     07:30:18      20:35:29        13:05                      
1     08:00:44      11:10:26        03:09                      
2     08:05:45      08:05:45        00:00                      
3     08:16:38      11:24:29        03:07                      
4     08:31:58      08:31:58        00:00                      

        NoPermissionReason                             StoreCheck-InImageLink  \
0                     None  https://lisimagenewv3.loveinstore.com//LISImag...   
1                     None  https://lisimagenewv3.loveinstore.com//LISImag...   
2  Permanent No Permission                                                      
3                     None  https://lisimagenewv3.loveinstore.com//LISImag...   
4         Permanent Closed                                                      

                             StoreCheck-OutImageLink FR Image  \
0  https://lisimagenewv3.loveinstore.com//LISImag...     None   
1  https://lisimagenewv3.loveinstore.com//LISImag...     None   
2                                                        None   
3  https://lisimagenewv3.loveinstore.com//LISImag...     None   
4                                                        None   

                               NoPermissionImageLink Covered Merchandised  
0                                               None       Y            Y  
1                                               None       Y            Y  
2  https://lisimagenewv3.loveinstore.com//LISImag...       Y            N  
3                                               None       Y            Y  
4  https://lisimagenewv3.loveinstore.com//LISImag...       Y            N

In [12]:
start_date = '2025-11-01'
end_date   = '2025-11-05'

In [13]:
PrimaryShelfQuery = f"""
SELECT DISTINCT
    VisitID,
    TRY_CONVERT(date, [SubmittedDate&Time], 105) AS SubmittedDate,
    Username,FullName,Designation,LISStoreCode,ProductCode,ProductName,
    Quantity,Amount,
    (CASE WHEN Quantity >=3  THEN 1 ELSE 0 END) AS OSA
FROM PowerBI_LISV2.DaburPharmaManagement.PrimaryShelfStockReportExport
WHERE 
    UserName NOT LIKE '%SUP%'
    AND LOWER(UserName) NOT LIKE '%test%'
    AND LOWER(UserName) NOT LIKE '%star%'
    AND [Month] = 11
    AND [Year] = 2025
    AND TRY_CONVERT(date, [SubmittedDate&Time], 105) >= '{start_date}'
    AND TRY_CONVERT(date, [SubmittedDate&Time], 105) <= '{end_date}'
ORDER BY 
    TRY_CONVERT(date, [SubmittedDate&Time], 105);
"""


In [14]:
PrimaryShelf = pd.read_sql(PrimaryShelfQuery,conn)

In [15]:
PrimaryShelf = PrimaryShelf.drop_duplicates()

In [16]:
PrimaryShelfOSA = (
    PrimaryShelf.groupby(["VisitID", "SubmittedDate", "Username", "LISStoreCode"], dropna=False)
      .agg(**{
          "OSA Target": ("OSA", "size"),
          "OSA Avail": ("OSA", "sum"),
      })
      .reset_index()
)

In [17]:
# Convert to datetime (auto-detects multiple formats)
PrimaryShelfOSA["SubmittedDate"] = pd.to_datetime(PrimaryShelfOSA["SubmittedDate"], errors="coerce")

# Format into dd-mm-yyyy string
PrimaryShelfOSA["SubmittedDate"] = PrimaryShelfOSA["SubmittedDate"].dt.strftime("%d-%m-%Y")

In [18]:
PrimaryShelfOSA.head()

VisitID SubmittedDate    Username          LISStoreCode  OSA Target  \
0   387634    01-11-2025  DBR-ISP148  DBRPH-MRG0GO00577577          40   
1   387635    01-11-2025  DBR-ISP239  DBRPH-TRU0TN00906372          60   
2   387636    01-11-2025  DBR-ISP266  DBRPH-JPR0RJ00570189          60   
3   387637    01-11-2025  DBR-ISP410  DBRPH-NDY0AP00945235          60   
4   387638    01-11-2025  DBR-ISP427  DBRPH-CTK0OR00099956          60   

   OSA Avail  
0         21  
1          7  
2          5  
3         28  
4         37

In [19]:
PrimaryShelfOSA.to_csv('PrimaryShelfOSA.csv')

In [20]:
StoreAttendance['VisitDate'] = pd.to_datetime(StoreAttendance['VisitDate'], errors='coerce', dayfirst=True).dt.normalize()
PrimaryShelfOSA['SubmittedDate'] = pd.to_datetime(PrimaryShelfOSA['SubmittedDate'], errors='coerce', dayfirst=True).dt.normalize()

In [21]:
# StoreAttendance.head()

In [22]:
pss1 = pd.merge(StoreAttendance,PrimaryShelfOSA[['SubmittedDate', 'Username', 'LISStoreCode','OSA Target','OSA Avail']], left_on =['VisitDate','UserName','LISStoreCode'], 
                right_on=['SubmittedDate', 'Username', 'LISStoreCode'], how = 'left')

In [23]:
pss1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3897 entries, 0 to 3896
Data columns (total 21 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   UserName                 3897 non-null   object        
 1   FullName                 3897 non-null   object        
 2   LISStoreCode             3897 non-null   object        
 3   StoreName                3897 non-null   object        
 4   VisitDate                3897 non-null   datetime64[ns]
 5   PJPStatus                3897 non-null   object        
 6   CheckIn-Time             3897 non-null   object        
 7   CheckOut-Time            3897 non-null   object        
 8   TimeDuration             3897 non-null   object        
 9   AutoCheckOutStatus       3897 non-null   object        
 10  NoPermissionReason       1306 non-null   object        
 11  StoreCheck-InImageLink   3897 non-null   object        
 12  StoreCheck-OutImageLink  3897 non-

In [24]:
pss1.to_csv('pss1.csv')

#### SOS ( Data Preparation , Unpivot, Pivot & ColumnReordering

In [25]:
SOSQuery =f"""
SELECT 
    VisitId,
    TRY_CONVERT(date, [SubmittedDate&Time], 105) AS SubmittedDate,
    UserName,FullName,LISStoreCode,IscategoryAvailable,ReasonforNonavailability,
    CategoryTrackingName,ISPCategoryTrackingProductCode,CategoryProductName,
    Facings,TotalFacings,FacingContribution,TgtCatSOS,
    ImageLink as 'URL'
FROM PowerBI_LISV2.DaburPharmaManagement.CategoryTrackingVisitExport
WHERE 
    UserName NOT LIKE '%SUP%'
    AND LOWER(UserName) NOT LIKE '%test%'
    AND LOWER(UserName) NOT LIKE '%star%'
    AND [Month] = 11
    AND [Year] = 2025
    AND TRY_CONVERT(date, [SubmittedDate&Time], 105) >= '{start_date}'
    AND TRY_CONVERT(date, [SubmittedDate&Time], 105) <= '{end_date}'
ORDER BY 
    TRY_CONVERT(date, [SubmittedDate&Time], 105);

"""

In [26]:
SOSData = pd.read_sql(SOSQuery,conn)

In [27]:
SOSData= SOSData.rename(columns = {'Facings':'DaburFacings'})

In [28]:
SOSData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15714 entries, 0 to 15713
Data columns (total 15 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   VisitId                         15714 non-null  int64 
 1   SubmittedDate                   15714 non-null  object
 2   UserName                        15714 non-null  object
 3   FullName                        15714 non-null  object
 4   LISStoreCode                    15714 non-null  object
 5   IscategoryAvailable             15714 non-null  object
 6   ReasonforNonavailability        3989 non-null   object
 7   CategoryTrackingName            15714 non-null  object
 8   ISPCategoryTrackingProductCode  15714 non-null  object
 9   CategoryProductName             15714 non-null  object
 10  DaburFacings                    11725 non-null  object
 11  TotalFacings                    15714 non-null  object
 12  FacingContribution              11545 non-null

In [29]:
# Correct filtering: one boolean condition inside one pair of brackets
mask = SOSData["TgtCatSOS"].notna() & (SOSData["TgtCatSOS"].astype(str).str.strip() != "")
SOSData = SOSData[mask]

In [30]:
cols_to_replace_null = ["TotalFacings", "DaburFacings", "FacingContribution"]
SOSData[cols_to_replace_null] = SOSData[cols_to_replace_null].fillna(0)

In [31]:
SOSData["FacingContribution"] = pd.to_numeric(SOSData["FacingContribution"], errors="coerce")
SOSData["TgtCatSOS"] = pd.to_numeric(SOSData["TgtCatSOS"], errors="coerce")

SOSData["IsSOSAchieved"] = np.where(
    SOSData["FacingContribution"] >= SOSData["TgtCatSOS"],
    1, 
    0
).astype("int64")

In [32]:
# SOSData["IscategoryAvailable"] = np.where(
#     SOSData["IscategoryAvailable"] == 1,
#     'YES', 
#     'NO'
# ).astype(str)

In [33]:
SOSData.columns

Index(['VisitId', 'SubmittedDate', 'UserName', 'FullName', 'LISStoreCode',
       'IscategoryAvailable', 'ReasonforNonavailability',
       'CategoryTrackingName', 'ISPCategoryTrackingProductCode',
       'CategoryProductName', 'DaburFacings', 'TotalFacings',
       'FacingContribution', 'TgtCatSOS', 'URL', 'IsSOSAchieved'],
      dtype='object')

In [34]:
SOSData.to_csv('SOSData.csv') # chek here for duplication at date x user x store x catetracking name - can user re submit the facing counts 

In [35]:
# Select only specific columns from your DataFrame
ColumnsForPivot = SOSData[
    [
        "SubmittedDate",
        "UserName",
        "LISStoreCode",
        "IscategoryAvailable",
        "CategoryTrackingName",
        "DaburFacings",
        "TotalFacings",
        "FacingContribution",
        "TgtCatSOS",
        "URL",
        "IsSOSAchieved"
    ]
]

Unpivoted_SOSData = pd.melt(
    ColumnsForPivot,
    id_vars=[
        "SubmittedDate",
        "UserName",
        "LISStoreCode",
        "CategoryTrackingName"
    ],
    var_name="Attribute",
    value_name="Value"
)



In [36]:
Unpivoted_SOSData = Unpivoted_SOSData.sort_values(
    by=["SubmittedDate", "UserName", "LISStoreCode", "CategoryTrackingName"]
).reset_index(drop=True)

In [37]:
# Create new column by combining CategoryTrackingName and Attribute with an underscore
Unpivoted_SOSData["Category_Attribute"] = (
    Unpivoted_SOSData["CategoryTrackingName"].astype(str).str.strip() + "_" +
    Unpivoted_SOSData["Attribute"].astype(str).str.strip()
)

In [38]:
Unpivoted_SOSData.head()

SubmittedDate    UserName          LISStoreCode CategoryTrackingName  \
0    2025-11-01  DBR-ISP101  DBRPH-ARA0TN00714625        Air Freshners   
1    2025-11-01  DBR-ISP101  DBRPH-ARA0TN00714625        Air Freshners   
2    2025-11-01  DBR-ISP101  DBRPH-ARA0TN00714625        Air Freshners   
3    2025-11-01  DBR-ISP101  DBRPH-ARA0TN00714625        Air Freshners   
4    2025-11-01  DBR-ISP101  DBRPH-ARA0TN00714625        Air Freshners   

             Attribute Value                 Category_Attribute  
0  IscategoryAvailable   YES  Air Freshners_IscategoryAvailable  
1         DaburFacings     3         Air Freshners_DaburFacings  
2         TotalFacings     4         Air Freshners_TotalFacings  
3   FacingContribution    75   Air Freshners_FacingContribution  
4            TgtCatSOS    31            Air Freshners_TgtCatSOS

In [39]:
Unpivoted_SOSData= Unpivoted_SOSData.drop(columns = ['CategoryTrackingName','Attribute'])

In [40]:
SOSDatadedup = Unpivoted_SOSData.drop_duplicates(
    subset=["SubmittedDate", "UserName", "LISStoreCode", "Category_Attribute"],
    keep="first"
)

In [41]:
Pivoted_SOS_Data = (
    SOSDatadedup.pivot_table(
        index=["SubmittedDate", "UserName", "LISStoreCode"],
        columns="Category_Attribute",
        values="Value",
        aggfunc="first"          
    )
    .reset_index()
)

In [42]:
Pivoted_SOS_Data.head()

Category_Attribute SubmittedDate    UserName          LISStoreCode  \
0                     2025-11-01  DBR-ISP101  DBRPH-ARA0TN00714625   
1                     2025-11-01  DBR-ISP101  DBRPH-ARA0TN00791552   
2                     2025-11-01  DBR-ISP101  DBRPH-ARA0TN00834008   
3                     2025-11-01  DBR-ISP102   DBRPH-KOL0WB1287163   
4                     2025-11-01  DBR-ISP103  DBRPH-GRG0HR00967656   

Category_Attribute Air Freshners_DaburFacings  \
0                                           3   
1                                           0   
2                                           3   
3                                           5   
4                                          24   

Category_Attribute Air Freshners_FacingContribution  \
0                                                75   
1                                                 0   
2                                               100   
3                                                71   
4                                                40   

Category_Attribute Air Freshners_IsSOSAchieved  \
0                                            1   
1                                            0   
2                                            1   
3                                            1   
4                                            1   

Category_Attribute Air Freshners_IscategoryAvailable Air Freshners_TgtCatSOS  \
0                                                YES                      31   
1                                                 NO                      31   
2                                                YES                      31   
3                                                YES                      42   
4                                                YES                      28   

Category_Attribute Air Freshners_TotalFacings  \
0                                           4   
1                                           0   
2                                           3   
3                                           7   
4                                          60   

Category_Attribute                                  Air Freshners_URL  ...  \
0                   https://lisimagenewv3.loveinstore.com//LISImag...  ...   
1                                                                 NaN  ...   
2                   https://lisimagenewv3.loveinstore.com//LISImag...  ...   
3                   https://lisimagenewv3.loveinstore.com//LISImag...  ...   
4                   https://lisimagenewv3.loveinstore.com//LISImag...  ...   

Category_Attribute Toilet & Utensil Cleaner_TgtCatSOS  \
0                                                  15   
1                                                  15   
2                                                  15   
3                                                  15   
4                                                  15   

Category_Attribute Toilet & Utensil Cleaner_TotalFacings  \
0                                                      3   
1                                                      0   
2                                                      3   
3                                                      6   
4                                                     22   

Category_Attribute                       Toilet & Utensil Cleaner_URL  \
0                   https://lisimagenewv3.loveinstore.com//LISImag...   
1                                                                 NaN   
2                   https://lisimagenewv3.loveinstore.com//LISImag...   
3                   https://lisimagenewv3.loveinstore.com//LISImag...   
4                   https://lisimagenewv3.loveinstore.com//LISImag...   

Category_Attribute Toothpaste_DaburFacings Toothpaste_FacingContribution  \
0                                        5                           100   
1                                        0                             0   
2                                   

In [43]:
# list(Pivoted_SOS_Data.columns)

In [45]:
# Pivoted_SOS_Data = Pivoted_SOS_Data[ordered_cols]

In [46]:
Pivoted_SOS_Data.head()

Category_Attribute SubmittedDate    UserName          LISStoreCode  \
0                     2025-11-01  DBR-ISP101  DBRPH-ARA0TN00714625   
1                     2025-11-01  DBR-ISP101  DBRPH-ARA0TN00791552   
2                     2025-11-01  DBR-ISP101  DBRPH-ARA0TN00834008   
3                     2025-11-01  DBR-ISP102   DBRPH-KOL0WB1287163   
4                     2025-11-01  DBR-ISP103  DBRPH-GRG0HR00967656   

Category_Attribute Air Freshners_DaburFacings  \
0                                           3   
1                                           0   
2                                           3   
3                                           5   
4                                          24   

Category_Attribute Air Freshners_FacingContribution  \
0                                                75   
1                                                 0   
2                                               100   
3                                                71   
4                                                40   

Category_Attribute Air Freshners_IsSOSAchieved  \
0                                            1   
1                                            0   
2                                            1   
3                                            1   
4                                            1   

Category_Attribute Air Freshners_IscategoryAvailable Air Freshners_TgtCatSOS  \
0                                                YES                      31   
1                                                 NO                      31   
2                                                YES                      31   
3                                                YES                      42   
4                                                YES                      28   

Category_Attribute Air Freshners_TotalFacings  \
0                                           4   
1                                           0   
2                                           3   
3                                           7   
4                                          60   

Category_Attribute                                  Air Freshners_URL  ...  \
0                   https://lisimagenewv3.loveinstore.com//LISImag...  ...   
1                                                                 NaN  ...   
2                   https://lisimagenewv3.loveinstore.com//LISImag...  ...   
3                   https://lisimagenewv3.loveinstore.com//LISImag...  ...   
4                   https://lisimagenewv3.loveinstore.com//LISImag...  ...   

Category_Attribute Toilet & Utensil Cleaner_TgtCatSOS  \
0                                                  15   
1                                                  15   
2                                                  15   
3                                                  15   
4                                                  15   

Category_Attribute Toilet & Utensil Cleaner_TotalFacings  \
0                                                      3   
1                                                      0   
2                                                      3   
3                                                      6   
4                                                     22   

Category_Attribute                       Toilet & Utensil Cleaner_URL  \
0                   https://lisimagenewv3.loveinstore.com//LISImag...   
1                                                                 NaN   
2                   https://lisimagenewv3.loveinstore.com//LISImag...   
3                   https://lisimagenewv3.loveinstore.com//LISImag...   
4                   https://lisimagenewv3.loveinstore.com//LISImag...   

Category_Attribute Toothpaste_DaburFacings Toothpaste_FacingContribution  \
0                                        5                           100   
1                                        0                             0   
2                                   

In [53]:
# Pivoted_SOS_Data.info()

In [48]:
sos_cols = [col for col in Pivoted_SOS_Data.columns if "IsSOSAchieved" in col]
Pivoted_SOS_Data["SumSOSAchieved"] = Pivoted_SOS_Data[sos_cols].sum(axis=1, skipna=True)

In [49]:
sos_cols = [col for col in Pivoted_SOS_Data.columns if "Iscategoryavailable" in col]
Pivoted_SOS_Data["SumIsCategoryAvailable"] = Pivoted_SOS_Data[sos_cols].sum(axis=1, skipna=True)

In [50]:
cols = list(Pivoted_SOS_Data.columns)
order_pattern = [
    'IscategoryAvailable',
    'DaburFacings',
    'TotalFacings',
    'FacingContribution',
    'TgtCatSOS',
    'IsSOSAchieved',
    'URL'
]
# Extract unique categories
categories = sorted({re.split(r'_', c, 1)[0] for c in cols if '_' in c})

# Build ordered list
ordered_cols = ['SubmittedDate', 'UserName', 'LISStoreCode','SumSOSAchieved','SumIsCategoryAvailable']
for cat in categories:
    for suffix in order_pattern:
        match = f"{cat}_{suffix}"
        if match in cols:
            ordered_cols.append(match)

In [51]:
Pivoted_SOS_Data = Pivoted_SOS_Data[ordered_cols]

In [54]:
Pivoted_SOS_Data.info() # SOS Data Cleaned and Reordered

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1962 entries, 0 to 1961
Data columns (total 75 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   SubmittedDate                                 1962 non-null   object 
 1   UserName                                      1962 non-null   object 
 2   LISStoreCode                                  1962 non-null   object 
 3   SumSOSAchieved                                1962 non-null   object 
 4   SumIsCategoryAvailable                        1962 non-null   float64
 5   Air Freshners_IscategoryAvailable             1962 non-null   object 
 6   Air Freshners_DaburFacings                    1962 non-null   object 
 7   Air Freshners_TotalFacings                    1962 non-null   object 
 8   Air Freshners_FacingContribution              1962 non-null   object 
 9   Air Freshners_TgtCatSOS                       1962 non-null   o

#### Combining pss1 with SOS Cleaned & Pivoted Dataset

In [57]:
Pivoted_SOS_Data['SubmittedDate'] = pd.to_datetime(Pivoted_SOS_Data['SubmittedDate'], errors='coerce', dayfirst=True).dt.normalize()

In [58]:
pss2 = pd.merge(pss1,Pivoted_SOS_Data,left_on = ['LISStoreCode', 'UserName', 'VisitDate'], 
                right_on=['LISStoreCode', 'UserName', 'SubmittedDate'],how='left')

In [66]:
# pss2.info()

#### Paid Visibility ( Bay Branding & Window Visibility ) 

In [85]:
WindowVisibilityQuery = """
SELECT *
FROM PowerBI_LISV2.DaburPharmaManagement.PaidVisibilityAuditInputReport
WHERE
  LOWER(UserName) NOT LIKE '%sup%'
  AND LOWER(UserName) NOT LIKE '%star%'
  AND PortalQuestions IS NOT NULL
  AND TRY_CONVERT(date, [SubmittedDate&Time], 105) >= '2025-11-01'  -- ISO is unambiguous
  AND TRY_CONVERT(date, [SubmittedDate&Time], 105) <=  '2025-11-05'  -- end date + 1 day
  ;
"""

In [86]:
WindowVisibilityData = pd.read_sql(WindowVisibilityQuery,conn)

In [87]:
WindowVisibilityData.to_csv('PaidVisibilityAuditInputReport.csv')